In [46]:
from Data import PKLS_FILES, get_min_descriptionsNorm_triples_relations
from utils.utils import read_cached_array
import torch


In [ ]:
k = 10
descs, triples, relations, aliases = get_min_descriptionsNorm_triples_relations(k)

In [67]:
list(descs.items())[0:4]

[('Q30',
  "The United States of America (USA), commonly known as the United States (U.S. or US) or America, is a country comprising 50 states, a federal district, five major self-governing territories, and various possessions. At 3.8 million sq mi (9.8 million km2), the United States is the world's third or fourth largest country by total area and is slightly smaller than the entire continent of Europe's 3.93 million sq mi (10.2 million km2). With a population of more than 327 million people, the U.S. is the third most populous country. Most of the country is located contiguously in North America between Canada and Mexico. The extremely diverse geography, climate, and wildlife of the United States make it one of the world's 17 megadiverse countries.Paleo-indians migrated from Siberia to the North American mainland at least 12,000 years ago. European colonization began in the 16th century. The United States emerged from the thirteen British colonies established along the East Coast. Fo

In [71]:
head = "Q851213"
descs[head]

'Electronic dance music (EDM), also known as dance music, club music, or simply dance, is a broad range of percussive electronic music genres made largely for nightclubs, raves and festivals. it is generally produced for playback by disc jockeys who create seamless selections of tracks, called a mix by segueing from one recording to another. EDM producers also perform their music live in a concert or festival setting in what is sometimes called a live PA. in Europe, EDM is more commonly called \'dance music\', or simply \'dance\'.in the late 1980s and early 1990s, following the emergence of raving, pirate radios and an upsurge of interest in club culture, EDM achieved widespread mainstream popularity in Europe. in the United States at that time, acceptance of dance culture was not universal; although both electro and Chicago house music were influential both in Europe and the United States, mainstream media outlets and the record industry remained openly hostile to it. There was also a

In [72]:
trpls = triples[head]
print(f"head description: {descs[head]}")
print(f"head aliases: {aliases[head]}")
for h,r,t in trpls:
    print(f"r: {r}, t: {t}")
    print(f"relation aliases: {relations[r]}")
    print(f"tail description: {descs[t]}")
    print(f"tail aliases: {aliases[t]}")
    print("\n\n")

head description: Electronic dance music (EDM), also known as dance music, club music, or simply dance, is a broad range of percussive electronic music genres made largely for nightclubs, raves and festivals. it is generally produced for playback by disc jockeys who create seamless selections of tracks, called a mix by segueing from one recording to another. EDM producers also perform their music live in a concert or festival setting in what is sometimes called a live PA. in Europe, EDM is more commonly called 'dance music', or simply 'dance'.in the late 1980s and early 1990s, following the emergence of raving, pirate radios and an upsurge of interest in club culture, EDM achieved widespread mainstream popularity in Europe. in the United States at that time, acceptance of dance culture was not universal; although both electro and Chicago house music were influential both in Europe and the United States, mainstream media outlets and the record industry remained openly hostile to it. The

In [61]:
from transformers import BertTokenizerFast

# 1) Load the fast tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

sentence = "Troy Evans (born February 16, 1948) is an American actor who …"

# 2) Tokenize and ask for offset mappings
enc = tokenizer(
    sentence,
    return_offsets_mapping=True,  # gives you char ↔ token spans
    padding=False,
    truncation=False
)

# enc is a dict with:
# - input_ids:      [101, 1234, 5678, …, 102]
# - token_type_ids: [0,   0,    0,  …,  0]
# - attention_mask: [1,   1,    1,  …,  1]
# - offset_mapping: [(0,0),(0,4),(4,5), …, (len, len)]
#   one tuple per token, giving the (start_char, end_char) in the original string

tokens = tokenizer.convert_ids_to_tokens(enc['input_ids'])

print(len(tokens))
print(tokens)
start_probs = torch.tensor([[.1,.9,.1,.1,.1,.1,.1,.1,.1,.1,.1,.1,.1,.1,.1,.1,.1]])
end_probs = torch.tensor([[.1,.1,.9,.1,.1,.1,.1,.1,.1,.1,.1,.1,.1,.1,.1,.1,.1]])
start_probs.shape



17
['[CLS]', 'Troy', 'Evans', '(', 'born', 'February', '16', ',', '1948', ')', 'is', 'an', 'American', 'actor', 'who', '…', '[SEP]']


torch.Size([1, 17])

In [41]:
from torch.nn.utils.rnn import pad_sequence

def extract_first_embeddings(token_embs, start_probs, end_probs, max_len, threshold=.6):
    """
        args:
            token_embs: sentence embeddings with shape (batch_size, seq_length, hidden_size)
            start_probs: probabilities that an entity is a start (start of subject for forward, start of object for backward) with shape (batch_size, seq_len)
            end_probs: probabilities that an entity is an end (end of subject for forward, end of object for backward) with shape (batch_size, seq_len)
            threshold: threshold that if the probability > threshold, then it is considered start or end of starting entity
        returns:
            padded_embs: these are padded embeddings of all subjects or objects with shape (B, max_ents, H)
            mask_embs: mask to show the padded embs (0 for padding)
            start_idxs:  list (len=batch_size) where each item is a list of tuples, each tuple (start_idx, end_idx) of the entities
    """
    batch_size, seq_len, hidden_size = token_embs.shape

    start_mask = start_probs > threshold
    end_mask = end_probs > threshold

    start_idxs = []
    all_ents_embs = []
    all_masks = []

    for sent_idx in range(batch_size):
        start_indices = torch.nonzero(start_mask[sent_idx], as_tuple=False).squeeze(-1)
        end_indices = torch.nonzero(end_mask[sent_idx], as_tuple=False).squeeze(-1)

        ents_embs = []
        idxs_sentence = []
        used_ends = set()

        for start_idx in start_indices.tolist():
            candidates = [end_idx for end_idx in end_indices.tolist() if end_idx >= start_idx and end_idx not in used_ends]
            if candidates:
                end_idx = candidates[0]
                used_ends.add(end_idx)
                idxs_sentence.append((start_idx, end_idx))

                # Compute average embedding
                ent_emb = (token_embs[sent_idx, start_idx] + token_embs[sent_idx, end_idx]) / 2.0
                ents_embs.append(ent_emb)

        start_idxs.append(idxs_sentence)

        if ents_embs:
            ent_tensor = torch.stack(ents_embs)
            all_ents_embs.append(ent_tensor)
            all_masks.append(torch.ones(ent_tensor.size(0), dtype=torch.bool, device=token_embs.device))
        else:
            all_ents_embs.append(torch.empty(0, hidden_size, device=token_embs.device))
            all_masks.append(torch.zeros(0, dtype=torch.bool, device=token_embs.device))

    # Pad sequences
    padded_embs = pad_sequence(all_ents_embs, batch_first=True, padding_value=0.0)
    mask_embs = pad_sequence(all_masks, batch_first=True, padding_value=False)

    if max_len is not None:
        # Pad or truncate embeddings
        curr_len = padded_embs.size(1)
        if curr_len < max_len:
            pad_size = max_len - curr_len
            padding = torch.zeros(padded_embs.size(0), pad_size, padded_embs.size(2), device=padded_embs.device)
            padded_embs = torch.cat([padded_embs, padding], dim=1)

            mask_padding = torch.zeros(mask_embs.size(0), pad_size, dtype=torch.bool, device=mask_embs.device)
            mask_embs = torch.cat([mask_embs, mask_padding], dim=1)

        elif curr_len > max_len:
            padded_embs = padded_embs[:, :max_len, :]
            mask_embs = mask_embs[:, :max_len]

    return padded_embs, mask_embs, start_idxs


In [42]:


def get_h_gs(sentences, tokenizer, model, max_length):
    encoded = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt", max_length=max_length)
    input_ids = encoded['input_ids']
    attention_mask = encoded['attention_mask']
    
    with torch.no_grad():
        bert_output = model(input_ids=input_ids, attention_mask=attention_mask)
        embeddings = bert_output.last_hidden_state # (batch_size, seq_len ,hidden_size)

    mask_expanded = attention_mask.unsqueeze(-1).expand(embeddings.size())

    masked_embeddings = embeddings * mask_expanded

    sum_embeddings = masked_embeddings.sum(dim=1)  # [batch, hidden_size]
    token_counts = mask_expanded.sum(dim=1)  # [batch, 1]
    token_counts = token_counts.clamp(min=1)
    return sum_embeddings / token_counts, embeddings



In [45]:
from transformers import BertModel, BertTokenizer

tokenizer =BertTokenizer.from_pretrained('bert-base-cased')
model = BertModel.from_pretrained('bert-base-cased')

_, embs = get_h_gs(sentence, tokenizer, model, max_length=128  )  #  h_gs (batch_size, hidden_size), embs (batch_size, seq_len, hidden_size)


print(f"self embs shape: {embs.shape}")

self embs shape: torch.Size([1, 17, 768])


In [62]:
_,_, subj_idxs = extract_first_embeddings(embs,start_probs, end_probs, 128, threshold=.5)
subj_idxs

[[(1, 2)]]

In [74]:
# pip install flashtext
from flashtext import KeywordProcessor

tail_aliases = [
  'Television actor', 'film or stage actor', 'Theatre actress', 
  'film actor', 'Actors and actresses', 'actor & actress', 
  'Television star', 'Film Actor', "actors", "actor"
]
sentence = ["bomba","telivision", "actor", "english", "and"]
# 1) build your keyword‐processor
kp = KeywordProcessor(case_sensitive=False, )
for alias in tail_aliases:
    kp.add_keyword(alias)

# 2) extract all matches with their char spans
matches = kp.extract_keywords(sentence, span_info=True)
# matches is a list of tuples: [(keyword, start_char, end_char), …]

matches

AttributeError: 'list' object has no attribute 'join'